**참고문헌: 핸즈온 머신러닝(2판), 올레리앙 제롱 지음, 박해선 옮김, 11장 – 심층 신경망 훈련하기**

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/handson-ml2/blob/master/11_training_deep_neural_networks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

# 설정

먼저 몇 개의 모듈을 임포트합니다. 맷플롯립 그래프를 인라인으로 출력하도록 만들고 그림을 저장하는 함수를 준비합니다. 또한 파이썬 버전이 3.5 이상인지 확인합니다(파이썬 2.x에서도 동작하지만 곧 지원이 중단되므로 파이썬 3을 사용하는 것이 좋습니다). 사이킷런 버전이 0.20 이상인지와 텐서플로 버전이 2.0 이상인지 확인합니다.

In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 배치 정규화

ELU(또는 다른 ReLU 변종)와 함께 He 초기화를 사용하면 훈련 초기 단계에서 그레이디언트 소실이나 폭주 문제를 크게 감소시킬 수 있지만, 훈련하는 동안 다시 발생하지 않으리란 보장은 없습니다.

2015년 한 논문에서 세르게이 이오페(Sergey Ioffe)와 치리슈티언 세게지(Christian Szegedy)가 그레이디언트 소실과 폭주 문제를 해결하기 위한 배치 정규화(batch normalization, BN) 기법(http://proceedings.mlr.press/v37/ioffe15.html)을 제안하였습니다. 이 기법은 각 층에서 활성화 함수를 통과하기 전이나 후에 모델에 연산을 하나 추가합니다. 이 연산은 단순하게 입력을 원점에 맞추고 정규화한 다음, 각 층에서 두개의 새로운 파라미터로 결과값의 스케일을 조정하고 이동 시킵니다. 하나는 스케일 조정에, 다른 하나는 이동에 사용합니다. 많은 경우 신경망의 첫 번째 층으로 배치 정규화를 추가하면 훈련 세트를 (예를 들어 StandardScaler를 사용하여) 표준화할 필요가 없습니다. 배치 정규화 층이 이런 역할을 대신합니다 (한 번에 하나의 배치만 처리하기 때문에 근사적입니다. 또한 입력 특성마다 스케일을 조정하고 이동할 수 있습니다.)

블로그 그림 참조:https://gaussian37.github.io/dl-concept-batchnorm/

입력 데이터를 원점에 맞추고 정규화하려면 알고리즘은 평균과 표준편차를 추정해야 합니다. 이를 위해 현재 미니배치에서 입력의 평균과 표준편차를 평가합니다 (그래서 이름이 배치 정규화입니다).


**Equation 11-3: Batch Normalization algorithm**

$
\begin{split}
1.\quad & \mathbf{\mu}_B = \dfrac{1}{m_B}\sum\limits_{i=1}^{m_B}{\mathbf{x}^{(i)}}\\
2.\quad & {\mathbf{\sigma}_B}^2 = \dfrac{1}{m_B}\sum\limits_{i=1}^{m_B}{(\mathbf{x}^{(i)} - \mathbf{\mu}_B)^2}\\
3.\quad & \hat{\mathbf{x}}^{(i)} = \dfrac{\mathbf{x}^{(i)} - \mathbf{\mu}_B}{\sqrt{{\mathbf{\sigma}_B}^2 + \epsilon}}\\
4.\quad & \mathbf{z}^{(i)} = \gamma \otimes \hat{\mathbf{x}}^{(i)} + \beta
\end{split}
$


이 알고리즘을 살펴 봅시다. 

*   ${\mathbf{\mu}}_B$는 미니배치 B에 대해 평가한 입력의 평균 벡터입니다 (입력마다 하나의 평균을 가집니다).
*   ${\sigma}_B$도 미니배치에 대해 평가한 입력의 표준편차 벡터입니다.
*   $m_B$는 미니배치에 있는 샘플 수 입니다.
*  $\hat{\mathbf{x}}^{(i)}$는 평균이 0이고 정규화된 샘플 i의 입력입니다.
*  $\gamma$는 층의 출력 스케일 파라미터 벡터입니다 (입력마다 하나의 스케일 파라미터가 있습니다)
*  $\otimes$는 원소별 곱셈(element-wise multiplication)입니다. 
* ${\beta}$는 층의 출력 이동 (오프셋) 파라미터 벡터입니다 (입력마다 하나의 스케일 파라미터가 있습니다). 각 입력은 해당 파라미터만큼 이동합니다.
* $\epsilon$은 분모가 0이 되는 것을 막기 위한 작은 숫자(전형적으로 10^-5)입니다. 이를 안전을 위한 항(smoothing term)이라고 합니다.
* $\mathbf{z}^{(i)}$는 배치 정규화 연산의 출력입니다. 즉, 입력의 스케일을 조정하고 이동시킨 것입니다.


훈련하는 동안 배치 정규화는 입력을 정규화한 다음 스케일을 조정하고 이동 시킵니다.테스트 시에는 어떻게 할까요? 샘플의 배치가 아니라 샘플 하나에 대한 예측을 만들어야 합니다. 이 경우 입력의 평균과 표준편차를 계산할 방법이 없습니다. 샘플의 배치를 사용한다 하더라도 매우 작거나 독립동일분포(independent indentically distributed)조건을 만족하지 못 할 수 있습니다. 이런 배치 샘플에서 배치 입력과 평균과 표쥰편차로 이 '최종'입력 평균과 표준편차를 대신 사용 할 수 있습니다. 그러나 대부분 배치 정규화 구현은 층의 입력 평균과 표준편차의 이동평균 (moving average)을 사용해 훈련하는 동안 최종 통계를 추정합니다.


케라스의 BatchNormalization층은 이를 자동으로 수행합니다. 정리하면 배치 정규화 층마다 네 개의 파라미터 벡터가 학습됩니다. $\gamma$(출력 스케일 벡터)와 ${\beta}$(출력 이동벡터) 는 일반적인 역전파를 통해 학습됩니다. ${\mathbf{\mu}}$(최종 입력 평균 벡터) 와 ${\sigma}$(최종 입력 퓨준편차 벡터)는 지수 이동 평균을 사용하여 추정됩니다. ${\mathbf{\mu}}$와 ${\sigma}$는 훈련하는 동안 추정되지만 훈련이 끝난 후에 사용됩니다([식 11-3]에 있는 배치 입력 평균과 표준편차를 대체하기 위해). 

해당 방법을 통해 이미지넷 분류 작업에서 큰 성과를 냈습니다. 그레이디언트 소실 문제가 크게 감소하여 하이퍼볼릭 탄젠트나 로지스틱 활성화 함수 같은 수렴성을 가진 활성화 함수를 사용할 수 있습니다. 또 가중치 초기화 에 네트워크가 훨씬 덜 민감해집니다.

그러나 배치 정규화는 모델의 복잡도를 키웁니다. 더군다나 실행 시간면에서도 손해입니다. 층마다 추가되는 계산이 신경망의 예측을 느리게 합니다. 다행히 훈련이 끝난 후에 이전층과 배치 정규화 층을 합쳐 실행 속도 저하를 피할 수 있습니다. 이전의 가중치를 바꾸어 바로 스케일이 조정되고 이동된 출력을 만듭니다.

예를 들면 이전 층이 $\mathbf{XW+b}$를 계산하면 배치 정규화 층은 $\gamma \otimes(\mathbf{XW+b-{\mu}})/{\sigma}+{\beta}$를 계산합니다(분오에 안전을 위해 추가하는 항인 $\epsilon$은 무시합니다). 만약 $\mathbf{W'= \gamma \otimes W'/{\sigma}}$와 $\mathbf{b'= \gamma \otimes (b - \mu)/{\sigma}+\beta}$를 정의하면 이식은  $\mathbf{XW'+b'}$로 단순화 됩니다. 따라서 이전 층의 가중치와 편향( $\mathbf{W}$ 와 $\mathbf{b}$)을 업데이트된 가중치와 편향($\mathbf{W'}$ 와 $\mathbf{b'}$)으로 바꾸면 배치 정규화층을 제거 할 수 있습니다. 

* 배치 정규화를 사용할 때 에포크마다 더 많은 시간이 걸리므로 훈련이 오히려 느려질 수 있습니다. 하지만 배치 정규화를 사용하면 수렴이 훨씬 빨라지므로 보통 상쇄됩니다. 따라서 더 적은 에포크로 동일한 성능에 도달할 수 있습니다. 대체로 실제 걸리는 시간은 보통 더 짧습니다. 

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)              4

In [ ]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
# updates 속성은 향후 삭제될 예정입니다.
# bn1.updates

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 15s 5ms/step - loss: 0.8912 - accuracy: 0.7006 - val_loss: 0.5663 - val_accuracy: 0.8126
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5808 - accuracy: 0.8012 - val_loss: 0.4809 - val_accuracy: 0.8380
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5168 - accuracy: 0.8207 - val_loss: 0.4419 - val_accuracy: 0.8482
Epoch 4/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4796 - accuracy: 0.8334 - val_loss: 0.4203 - val_accuracy: 0.8556
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4562 - accuracy: 0.8407 - val_loss: 0.4041 - val_accuracy: 0.8608
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4409 - accuracy: 0.8461 - val_loss: 0.3901 - val_accuracy: 0.8632
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4240 - accuracy: 0.8492 - val_loss: 0.3827 - val_accurac

이따금 활성화 함수전에 BN을 적용해도 잘 동작합니다(여기에는 논란의 여지가 있습니다). 또한 `BatchNormalization` 층 이전의 층은 편향을 위한 항이 필요 없습니다. `BatchNormalization` 층이 이를 무효화하기 때문입니다. 따라서 필요 없는 파라미터이므로 `use_bias=False`를 지정하여 층을 만들 수 있습니다:

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization 
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 1.0361 - accuracy: 0.6784 - val_loss: 0.6752 - val_accuracy: 0.7910
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.6807 - accuracy: 0.7831 - val_loss: 0.5563 - val_accuracy: 0.8192
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5929 - accuracy: 0.8065 - val_loss: 0.4988 - val_accuracy: 0.8360
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5488 - accuracy: 0.8181 - val_loss: 0.4648 - val_accuracy: 0.8450
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5159 - accuracy: 0.8258 - val_loss: 0.4412 - val_accuracy: 0.8534
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4939 - accuracy: 0.8325 - val_loss: 0.4242 - val_accuracy: 0.8588
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4721 - accuracy: 0.8391 - val_loss: 0.4102 - val_accuracy:

성능 비교를 위해서 동일한 모델을 Batch Normalization 없이 훈련을 해보겠습니다.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
#    keras.layers.BatchNormalization(),
    keras.layers.Dense(300),
# keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100),
# keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.3777 - accuracy: 0.6023 - val_loss: 0.9190 - val_accuracy: 0.7074
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.8237 - accuracy: 0.7266 - val_loss: 0.7332 - val_accuracy: 0.7622
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7022 - accuracy: 0.7704 - val_loss: 0.6497 - val_accuracy: 0.7930
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6363 - accuracy: 0.7926 - val_loss: 0.5972 - val_accuracy: 0.8082
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5927 - accuracy: 0.8057 - val_loss: 0.5629 - val_accuracy: 0.8202
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5616 - accuracy: 0.8144 - val_loss: 0.5357 - val_accuracy: 0.8284
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5381 - accuracy: 0.8203 - val_loss: 0.5151 - val_accuracy: